# STEP1: IDENTIFYING PROBLEM STATEMENT

Many social programs have a hard time ensuring that the right people are given enough aid. It's tricky when a program focuses on the poorest segment of the population. This segment of the population can't provide the necessary income and expense records to prove that they qualify.

In Latin America, a popular method called "Proxy Means Test" uses an algorithm to verify Income Qualification. With PMT, agencies use a model that considers the family's observable household attributes like the material of the walls and ceiling or the assests in their homes to classify them and predict their level of need.

While this is an improvement, accuracy remains a problem as the region's population grows and poverty decilnes.

The Inter-American Development Bank also believes that new methods beyond traditional econometrics, based on a dataset of Costa Rican household characteristics, might help improve PMT's performance.

### Following actions should be performed:

1. Identify the output variable.
2. Understand the type of data.
3. Check if there are any biases in your dataset.
4. Check whether all members of the house have the same poverty level.
5. Check if there is a house without a family head.
6. Set poverty level of the members and the head of the house within a family.
7. Count how many null values are existing in columns.
8. Remove null value rows of the target variable.
9. Predict the accuracy using random forest classifier.
10. Check the accuracy using random forest with cross validation.

#  ============================================================


### After reading the Problem Statement itself, we should be able to find what kind of problem statement it was all about.

This is a supervised multi-class classification machine learning problem:

* Supervised: provided with the labels for the training data
* Multi-class classification: Labels are discrete values with 4 classes

Each row represents one individual and each column is a feature, either unique to the individual, or for the household of the individual. The training set has one additional column, Target, which represents the poverty level on a 1-4 scale and is the label for the competition. A value of 1 is the most extreme poverty.

# STEP 2: UNDERSTANDING THE WEIGHTAGE OF PROBLEM STATEMENT  AND CONVERTING THE BUSINESS PROBLEM INTO A DATA SCIENCE PROBLEM STATEMENT

Before we look into the dataset, its the responsibility of every Data scientists to go to the environment to understand the exact weightage of the Problem Statement. Today we all have everything at our fingertips. If we just google for something, it's going to give the complete history. So, first let's understand "What is Costa Rica all about? What is IDB? What is Proxmy Means Test? What is the Focus of majority of the social programs to help Costa Rica?"

Let's understand now. 

## What I have to do?

I need to identify the "level of income qualification" needed for the families in Latin America and also to maximize the Accuracy score across all the categories. A generalized solution would be helpful to IDB and other institutions working towards helping the economically weaker sections of the society.

# STEP3: IDENTIFYING DATA SOURCES AND DATA UNDERSTANDING 

### 3.1 importing necessary libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### 3.2 Importing the Dataset

In [3]:
test_df = pd.read_csv("./data/test.csv")
train_df = pd.read_csv("./data/train.csv")

### 3.3 Exploring Train and Test dataset

In [4]:
print("Train Dataset - Rows,Columns:",train_df.shape)
print("Test Dataset - Rows,Columns:",test_df.shape)


Train Dataset - Rows,Columns: (9557, 143)
Test Dataset - Rows,Columns: (23856, 142)


In [5]:
train_df.head()

,Id,v2a1,hacdor,rooms,hacapo,v14a,refrig,v18q,v18q1,r4h1,...,SQBescolari,SQBage,SQBhogar_total,SQBedjefe,SQBhogar_nin,SQBovercrowding,SQBdependency,SQBmeaned,agesq,Target
0,ID_279628684,190000.0,0,3,0,1,1,0,NaN,0,...,100,1849,1,100,0,1.000000,0.0,100.0,1849,4
1,ID_f29eb3ddd,135000.0,0,4,0,1,1,1,1.0,0,...,144,4489,1,144,0,1.000000,64.0,144.0,4489,4
2,ID_68de51c94,NaN,0,8,0,1,1,0,NaN,0,...,121,8464,1,0,0,0.250000,64.0,121.0,8464,4
3,ID_d671db89c,180000.0,0,5,0,1,1,1,1.0,0,...,81,289,16,121,4,1.777778,1.0,121.0,289,4
4,ID_d56d6f5f5,180000.0,0,5,0,1,1,1,1.0,0,...,121,1369,16,121,4,1.777778,1.0,121.0,1369,4


In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9557 entries, 0 to 9556
Columns: 143 entries, Id to Target
dtypes: float64(8), int64(130), object(5)
memory usage: 10.4+ MB


#### Observation

In [7]:
train_df['Target'].value_counts()

4    5996
2    1597
3    1209
1     755
Name: Target, dtype: int64

top column which have missing values


,0
rez_esc,7928
v18q1,7342
v2a1,6860
SQBmeaned,5
meaneduc,5


rez_esc : Years behind in school         
v18ql : Number of tablets houshold owns            
v2a1 : Monthly rent payment             
meaneduc : Average years of education for adults(18+)

Overall Percentage of null values column WindowsSelectorEventLoopPolicy


### 3.4 Understanding the column variable

In [9]:
# check the datatype
# integer type columns
print("integer type")
train_df.select_dtypes(int).columns

integer type


Index(['hacdor', 'rooms', 'hacapo', 'v14a', 'refrig', 'v18q', 'r4h1', 'r4h2',
       'r4h3', 'r4m1',
       ...
       'area1', 'area2', 'age', 'SQBescolari', 'SQBage', 'SQBhogar_total',
       'SQBedjefe', 'SQBhogar_nin', 'agesq', 'Target'],
      dtype='object', length=130)

In [10]:
# integer type columns
print("integer type")
train_df.select_dtypes(float).columns

integer type


Index(['v2a1', 'v18q1', 'rez_esc', 'meaneduc', 'overcrowding',
       'SQBovercrowding', 'SQBdependency', 'SQBmeaned'],
      dtype='object')

In [11]:
# object type columns
print("object type columns")
train_df.select_dtypes(object).columns

object type columns


Index(['Id', 'idhogar', 'dependency', 'edjefe', 'edjefa'], dtype='object')

In [14]:
# check object data typew
print(train_df['dependency'].unique())

['no' '8' 'yes' '3' '.5' '.25' '2' '.66666669' '.33333334' '1.5'
 '.40000001' '.75' '1.25' '.2' '2.5' '1.2' '4' '1.3333334' '2.25'
 '.22222222' '5' '.83333331' '.80000001' '6' '3.5' '1.6666666' '.2857143'
 '1.75' '.71428573' '.16666667' '.60000002']


## STEP 4: EXPLORATORY DATA ANALYSIS(EDA)

### DATA CLEANING - DATA PREPARATION - DATA TRANSFORMATION

### 4.1 Checking missing values for Entire train dataset

In [27]:
# step 1 check null values
print("top column which have missing values")
missmap=train_df.isna().sum().to_frame()
missmap = missmap.sort_values(0,ascending=False)
missmap.head()

top column which have missing values


,0
rez_esc,7928
v18q1,7342
v2a1,6860
SQBmeaned,5
meaneduc,5


##### 4.1.1 Overall Percentage of null values column-wise

In [32]:
per_null_train = train_df.isnull().sum().sort_values(axis=0,ascending=False)/(train_df.shape[0]) * 100
print(per_null_train.head())

rez_esc      82.954902
v18q1        76.823271
v2a1         71.779847
SQBmeaned     0.052318
meaneduc      0.052318
dtype: float64


### 4.2 Data Preparation


Let's apply our commonsense for Data Cleaning. Data Cleaning has to to be approached by looking at the datatypes first of all.
Here we are able to see 5 columns with null values in the entire dataset. Let's look into the datatypes and apply our commonsense to fill the null values to transform the dataset in a proper form and shape.

Fields with missing values:

     rez_esc    =  Years behind in school

     v18q1      =  Number of tablets household owns

     v2a1       =  Monthly rent paymen
    
     meaneduc   =  Average years of education for adults (18+)

     SQBmeaned  =  Square of the mean years of education of adults (>=18) in the household

##### 4.2.1 Let's Explore the null values with respect to data types

4.2.1.1 Any null values in integer datatypes variable?


In [35]:
train_df.select_dtypes(int).head()

,hacdor,rooms,hacapo,v14a,refrig,v18q,r4h1,r4h2,r4h3,r4m1,...,area1,area2,age,SQBescolari,SQBage,SQBhogar_total,SQBedjefe,SQBhogar_nin,agesq,Target
0,0,3,0,1,1,0,0,1,1,0,...,1,0,43,100,1849,1,100,0,1849,4
1,0,4,0,1,1,1,0,1,1,0,...,1,0,67,144,4489,1,144,0,4489,4
2,0,8,0,1,1,0,0,0,0,0,...,1,0,92,121,8464,1,0,0,8464,4
3,0,5,0,1,1,1,0,2,2,1,...,1,0,17,81,289,16,121,4,289,4
4,0,5,0,1,1,1,0,2,2,1,...,1,0,37,121,1369,16,121,4,1369,4


In [36]:
null_counts = train_df.select_dtypes(int).isnull().sum()
null_counts[null_counts>0]

Series([], dtype: int64)

##### 4.2.1.2 any null value in float datatype variable?


In [39]:
train_df.select_dtypes(float).head()

,v2a1,v18q1,rez_esc,meaneduc,overcrowding,SQBovercrowding,SQBdependency,SQBmeaned
0,190000.0,NaN,NaN,10.0,1.000000,1.000000,0.0,100.0
1,135000.0,1.0,NaN,12.0,1.000000,1.000000,64.0,144.0
2,NaN,NaN,NaN,11.0,0.500000,0.250000,64.0,121.0
3,180000.0,1.0,1.0,11.0,1.333333,1.777778,1.0,121.0
4,180000.0,1.0,NaN,11.0,1.333333,1.777778,1.0,121.0


In [41]:
null_counts = train_df.select_dtypes(float).isnull().sum()
null_counts[null_counts > 0]

v2a1         6860
v18q1        7342
rez_esc      7928
meaneduc        5
SQBmeaned       5
dtype: int64

##### 4.2.1.3 check null values in object datatype variable

In [42]:
train_df.select_dtypes(object).head()

,Id,idhogar,dependency,edjefe,edjefa
0,ID_279628684,21eb7fcc1,no,10,no
1,ID_f29eb3ddd,0e5d7a658,8,12,no
2,ID_68de51c94,2c7317ea8,8,no,11
3,ID_d671db89c,2b58d945f,yes,11,no
4,ID_d56d6f5f5,2b58d945f,yes,11,no


In [43]:
# find the columns with null values
null_counts = train_df.select_dtypes(object).isna().sum()
null_counts[null_counts >0]

Series([], dtype: int64)

>>> OBSERVATION:

Looking at the different types of data and null values for each feature. We found the following: 

1. No null values for Integer type features. 

2. No null values for Object type features. 

3. But for Float types, we have observed the null values. And they are 


     rez_esc    =  Years behind in school

     v18q1      =  Number of tablets household owns

     v2a1       =  Monthly rent payment
    
     meaneduc   =  Average years of education for adults (18+)

     SQBmeaned  =  Square of the mean years of education of adults (>=18) in the household

### 4.3 Data Transformation

 Data Transformation

Motive : To make the entire dataset in a proper form and shape to fed into the model for best prediction.

#### Before treating the null value, it is good to check is there any mixed values in any column. It is known that if there is any mixed values in should be in 'object datatype'.

The object datatypes are :

['Id', 'idhogar', 'dependency', 'edjefe', 'edjefa']

In [48]:
# Object datatypes:

print('Object Type: ')
print(train_df.select_dtypes(object).columns)

Object Type: 
Index(['Id', 'idhogar', 'dependency', 'edjefe', 'edjefa'], dtype='object')


##### 4.3.1 Let's also fix the column with mixed values,if there's any.


In [51]:
train_df['Id'].value_counts()

ID_279628684    1
ID_27e24f23b    1
ID_8f7111b02    1
ID_0f06819f3    1
ID_730e245a6    1
               ..
ID_f78d4ee67    1
ID_984d7e9a9    1
ID_08d8e727d    1
ID_1c880df2e    1
ID_a38c64491    1
Name: Id, Length: 9557, dtype: int64

In [53]:
print(train_df['idhogar'].value_counts())

fd8a6d014    13
ae6cf0558    12
0c7436de6    12
b7a0b59d7    11
4476ccd4c    11
             ..
9f6b2b309     1
75226a178     1
1637ac45b     1
0194d569d     1
21eb7fcc1     1
Name: idhogar, Length: 2988, dtype: int64


In [54]:
train_df['dependency'].value_counts()

yes          2192
no           1747
.5           1497
2             730
1.5           713
.33333334     598
.66666669     487
8             378
.25           260
3             236
4             100
.75            98
.2             90
.40000001      84
1.3333334      84
2.5            77
5              24
1.25           18
3.5            18
.80000001      18
2.25           13
.71428573      12
1.75           11
1.2            11
.83333331      11
.22222222      11
.2857143        9
1.6666666       8
.60000002       8
6               7
.16666667       7
Name: dependency, dtype: int64

In [55]:
train_df['edjefe'].value_counts()

no     3762
6      1845
11      751
9       486
3       307
15      285
8       257
7       234
5       222
14      208
17      202
2       194
4       137
16      134
yes     123
12      113
10      111
13      103
21       43
18       19
19       14
20        7
Name: edjefe, dtype: int64

In [56]:
train_df['edjefa'].value_counts()

no     6230
6       947
11      399
9       237
8       217
15      188
7       179
5       176
3       152
4       136
14      120
16      113
10       96
2        84
17       76
12       72
yes      69
13       52
21        5
19        4
18        3
20        2
Name: edjefa, dtype: int64

**`Observation:`**

Looking at the above results, we are able to see that there are 3 Object datatypes with mixed values. And they are,

['dependency','edjefe','edjefa']


According to the documentation for these columns:

* dependency: Dependency rate, calculated = (number of members of the household younger than 19 or older than 64)/(number of member of household between 19 and 64)


* edjefe: years of education of male head of household, based on the interaction of escolari (years of education), head of household and gender, yes=1 and no=0


* edjefa: years of education of female head of household, based on the interaction of escolari (years of education), head of household and gender, yes=1 and no=0

For these three variables, it seems “yes” = 1 and “no” = 0. We can correct the variables using a mapping and convert to floats.

In [57]:
mapping = {'yes':1,'no':0}

for df in [train_df,test_df]:
    df['dependency'] = df['dependency'].replace(mapping).astype(np.float64)
    df['edjefe'] = df['edjefe'].replace(mapping).astype(np.float64)
    df['edjefa'] = df['edjefa'].replace(mapping).astype(np.float64)


In [59]:
# recheck once for object datattype
print('Object datatype')
print(train_df.select_dtypes(object).columns)

Object datatype
Index(['Id', 'idhogar'], dtype='object')


In [60]:
pd.set_option("max_columns",None)

In [61]:
train_df.describe()

,v2a1,hacdor,rooms,hacapo,v14a,refrig,v18q,v18q1,r4h1,r4h2,r4h3,r4m1,r4m2,r4m3,r4t1,r4t2,r4t3,tamhog,tamviv,escolari,rez_esc,hhsize,paredblolad,paredzocalo,paredpreb,pareddes,paredmad,paredzinc,paredfibras,paredother,pisomoscer,pisocemento,pisoother,pisonatur,pisonotiene,pisomadera,techozinc,techoentrepiso,techocane,techootro,cielorazo,abastaguadentro,abastaguafuera,abastaguano,public,planpri,noelec,coopele,sanitario1,sanitario2,sanitario3,sanitario5,sanitario6,energcocinar1,energcocinar2,energcocinar3,energcocinar4,elimbasu1,elimbasu2,elimbasu3,elimbasu4,elimbasu5,elimbasu6,epared1,epared2,epared3,etecho1,etecho2,etecho3,eviv1,eviv2,eviv3,dis,male,female,estadocivil1,estadocivil2,estadocivil3,estadocivil4,estadocivil5,estadocivil6,estadocivil7,parentesco1,parentesco2,parentesco3,parentesco4,parentesco5,parentesco6,parentesco7,parentesco8,parentesco9,parentesco10,parentesco11,parentesco12,hogar_nin,hogar_adul,hogar_mayor,hogar_total,dependency,edjefe,edjefa,meaneduc,instlevel1,instlevel2,instlevel3,instlevel4,instlevel5,instlevel6,instlevel7,instlevel8,instlevel9,bedrooms,overcrowding,tipovivi1,tipovivi2,tipovivi3,tipovivi4,tipovivi5,computer,television,mobilephone,qmobilephone,lugar1,lugar2,lugar3,lugar4,lugar5,lugar6,area1,area2,age,SQBescolari,SQBage,SQBhogar_total,SQBedjefe,SQBhogar_nin,SQBovercrowding,SQBdependency,SQBmeaned,agesq,Target
count,2.697000e+03,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,2215.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,1629.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.00000,9557.000000,9557.000000,9557.0,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9552.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.00000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9552.000000,9557.000000,9557.000000
mean,1.652316e+05,0.038087,4.955530,0.023648,0.994768,0.957623,0.231767,1.404063,0.385895,1.559171,1.945066,0.399184,1.661714,2.060898,0.785079,3.220885,4.005964,3.999058,4.094590,7.200272,0.459791,3.999058,0.594015,0.077744,0.188030,0.008580,0.115622,0.013079,0.001465,0.001465,0.692791,0.222873,0.000942,0.001046,0.016428,0.065920,0.970074,0.017683,0.003139,0.002197,0.677409,0.964738,0.031705,0.003558,0.885110,0.000314,0.002197,0.110809,0.003872,0.213979,0.764257,0.015381,0.002511,0.001883,0.489589,0.458407,0.050120,0.882704,0.03003,0.084545,0.001465,0.0,0.001256,0.102438,0.327404,0.570158,0.128388,0.288061,0.583551,0.101078,0.252799,0.646123,0.057549,0.483415,0.516585,0.137805,0.123260,0.268390,0.031914,0.062781,0.033169,0.342681,0.311081,0.184054,0.381814,0.012138,0.009522,0.051167,0.010045,0.002407,0.013289,0.003139,0.012661,0.008685,1.406613,2.592445,0.284085,3.999058,1.149550,5.096788,2.896830,9.231523,0.134666,0.170556,0.20

#### 4.3.2 Lets fix the column with null values

#####  4.3.2.1 Null value treatment for `rez_esc` column - years behind school

In [62]:
train_df['rez_esc'].value_counts()

0.0    1211
1.0     227
2.0      98
3.0      55
4.0      29
5.0       9
Name: rez_esc, dtype: int64

In [78]:
train_df[train_df['rez_esc']==0]['age']

6        8
7        7
38      15
42      15
48       8
        ..
9514     8
9516    12
9533    17
9542    10
9543    13
Name: age, Length: 1211, dtype: int64

>'rez_esc'  - years behind in school. For the families with a null value, is possible that they have no children currently in school. Let's test this out by finding the ages of those who have a missing value in this column and the ages of those who do not have a missing value.

In [66]:
train_df[train_df['rez_esc'].notnull()]['age'].describe()

count    1629.000000
mean       12.258441
std         3.218325
min         7.000000
25%         9.000000
50%        12.000000
75%        15.000000
max        17.000000
Name: age, dtype: float64

>From the above , we see that when min age is 7 and max age 17 for years , then the behind in school column has value

In [70]:
train_df.loc[train_df['rez_esc'].isnull()]['age'].describe()

count    7928.000000
mean       38.833249
std        20.989486
min         0.000000
25%        24.000000
50%        38.000000
75%        54.000000
max        97.000000
Name: age, dtype: float64

**Statement:**

Anyone younger than 7 or older than 17 is presumably has no years behind and therefore the value should be set to 0. For this variable, if the individual is over 19 and they have a missing value, or if they are younger than 7 and have a missing value we can set it to zero.

Cross Checking once if there's any null value when the age is between 7 and 17 with respect to null values of 'years behind school' column

In [82]:
train_df.loc[train_df['rez_esc'].isnull() & ((train_df['age']>7) & (train_df['age']<17))]['age'].describe()

count     1.0
mean     10.0
std       NaN
min      10.0
25%      10.0
50%      10.0
75%      10.0
max      10.0
Name: age, dtype: float64

There is one value that has Null for the 'behind in school' column with age between 7 and 17 with the age 10. Let's find the
complete record of this household.

In [83]:
train_df[(train_df['age']==10) & train_df['rez_esc'].isnull()]

,Id,v2a1,hacdor,rooms,hacapo,v14a,refrig,v18q,v18q1,r4h1,r4h2,r4h3,r4m1,r4m2,r4m3,r4t1,r4t2,r4t3,tamhog,tamviv,escolari,rez_esc,hhsize,paredblolad,paredzocalo,paredpreb,pareddes,paredmad,paredzinc,paredfibras,paredother,pisomoscer,pisocemento,pisoother,pisonatur,pisonotiene,pisomadera,techozinc,techoentrepiso,techocane,techootro,cielorazo,abastaguadentro,abastaguafuera,abastaguano,public,planpri,noelec,coopele,sanitario1,sanitario2,sanitario3,sanitario5,sanitario6,energcocinar1,energcocinar2,energcocinar3,energcocinar4,elimbasu1,elimbasu2,elimbasu3,elimbasu4,elimbasu5,elimbasu6,epared1,epared2,epared3,etecho1,etecho2,etecho3,eviv1,eviv2,eviv3,dis,male,female,estadocivil1,estadocivil2,estadocivil3,estadocivil4,estadocivil5,estadocivil6,estadocivil7,parentesco1,parentesco2,parentesco3,parentesco4,parentesco5,parentesco6,parentesco7,parentesco8,parentesco9,parentesco10,parentesco11,parentesco12,idhogar,hogar_nin,hogar_adul,hogar_mayor,hogar_total,dependency,edjefe,edjefa,meaneduc,instlevel1,instlevel2,instlevel3,instlevel4,instlevel5,instlevel6,instlevel7,instlevel8,instlevel9,bedrooms,overcrowding,tipovivi1,tipovivi2,tipovivi3,tipovivi4,tipovivi5,computer,television,mobilephone,qmobilephone,lugar1,lugar2,lugar3,lugar4,lugar5,lugar6,area1,area2,age,SQBescolari,SQBage,SQBhogar_total,SQBedjefe,SQBhogar_nin,SQBovercrowding,SQBdependency,SQBmeaned,agesq,Target
2514,ID_f012e4242,160000.0,0,6,0,1,1,1,1.0,0,1,1,1,1,2,1,2,3,3,3,0,NaN,3,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,1,1,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0369a5d78,1,2,0,3,0.5,11.0,0.0,13.5,1,0,0,0,0,0,0,0,0,2,1.5,0,1,0,0,0,1,0,1,2,1,0,0,0,0,0,1,0,10,0,100,9,121,1,2.25,0.25,182.25,100,4


In [84]:
# from above we see taht teh 'behind in school' column has null values
# let's use the above to fix the data
for df in [train_df,test_df]:
    df['rez_esc'].fillna(value=0,inplace=True)

train_df['rez_esc'].isnull().sum()


0

##### 4.3.2.2 Null Value treatment for `v18q1` column (total nulls: 7342) -  Number of tablets household owns

It is fair to say that if a household owns a tablet, then the count would be some number, if not the count would be zero.

In our dataset, we have column 'v18q' which indicates - whether or not household owns a tablet.

Since, we are seeing the things from the household perspective, it only makes sense to look at it on a household level, so we'll only select the rows for the head of household. 

* 'parentesco1', =1 if household head

In [89]:
# Heads of household
heads = train_df.loc[train_df['parentesco1']==1].copy()
heads.groupby('v18q')['v18q1'].apply(lambda x: x.isnull().sum())

v18q
0    2318
1       0
Name: v18q1, dtype: int64